In [1]:
import pandas as pd
import win32com.client as win32
import json
import re
import os 
import datetime as dt

PATH = os.getcwd()

In [2]:
workbook_name = '\sample.xlsx' # Name of your excel file, in current working dir
df = pd.read_excel(PATH+workbook_name)
df.head()

,Type,Domain,SamAccountName,DisplayName,Description,Previous Owner employee ID,Previous Owner Name,Email,Service Account Still Required? (Y/N),Updated Owner Name,Remarks (if any),Interactive Longon Required? (Y/N),ISEN Ref #,Updated Owner Employee ID
0,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN,NaN
1,description-app,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN,NaN
2,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN,NaN
3,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN,NaN
4,interactive-logon,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
RECIPIENTS = [] # Receivers

CCs = ['janelle.tang@au.ey.com', # CCed Recipients
      'janelle.tang@au.ey.com'] 

def convert_list_to_recipients(lst):
    result = ""
    for email in lst:
        result = result + email + ';'
    return result

def create_mail(text, recipient_name, data, subject, recipient_email, location = None, app = None, attachment = False,send=False,):
    """
    @params:
    
    """
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient_email
    mail.CC = convert_list_to_recipients(CCs)
    mail.Subject = subject
    
    # fill in name
    recipient_name = recipient_name.replace('-',' ').title() 
    text = text.replace('{name}',recipient_name) 
    
    # fill in date
    future_date = (dt.datetime.now().date()+dt.timedelta(days=3)).strftime("%d-%b-%Y")
    text = text.replace('{future date}', future_date)
    
#     # fill in location
#     if app:
#         text = text.replace('{application}')
    
#     # fill in app
#     if location:
#         text = text.replace('{location}')
    
    if attachment:
        attachment_path = PATH+r'\attachments\{}'.format(data)
        mail.HtmlBody = text.replace('{name}',recipient_name)
        mail.Attachments.Add(attachment_path)
    else:
        
        ## Remove Null values
        data = data.replace('nan','')
        
        ## Styling
        data = data.replace('<table','<table border = 1 class="dataframe"')
        mail.HtmlBody = text.replace('{table}',data)

    if send:
        mail.send()
    else:
        mail.save()

def get_email_content(attachment,email_type):
    if attachment:
        d = text_dict[email_type]['attachment']
    else:
        d = text_dict[email_type]['no-attachment']
    body = d['body']
    subject = d['subject']
    return body, subject

Just a note - <br/><br/> I have kept the <b>attachments</b> in an attachment folder in the current working directory. 

In [21]:
## Loads email body text as a dictionary
with open('email_text.json') as f:
    text_dict = json.load(f)

email_list = df.Email.unique().tolist()
for email in email_list:
    data = df[df.Email == email] # Select all rows in data to be sent to this person
    name = email.split("@")[0].replace(".","-") # Get name from email address
    
    
    email_type_list = data.Type.unique()
    for email_type in email_type_list: # Split the types of emails to send (types of Orphan Accounts)
        
        ## Highlighted columns for recipient to fill - in the email_text.JSON
        required_cols = text_dict[email_type]['attachment']['columns']
        required_cols = required_cols.split(", ")
        
        select_cols = ['Domain',
                     'SamAccountName',
                     'DisplayName',
                     'Description',
                     'Previous Owner employee ID',
                     'Previous Owner Name'] + required_cols
        
        
        table = data[data.Type == email_type][select_cols]
        n_rows = table.shape[0]
        table.reset_index(drop=True)
        table = table.style.set_properties(**{'background-color': '#ffffbf','border':' 1pt solid #808080'}, subset=required_cols)
        if n_rows > 5:
            """
            If more than 5 rows, create an attachment file
            """
            
            body, subject = get_email_content(True,email_type)
            
            ## name of attachment
            attachment_name = "{}_{}.xlsx".format(email_type,name)
            table.to_excel('attachments/{}'.format(attachment_name),index=False)
            
            create_mail(body,name, attachment_name, subject, email, attachment = True)
            
        elif n_rows > 0:
            """
            Otherwise, just keep it as a table
            """
            html = table.render(index=False)
            body, subject = get_email_content(False,email_type)
            create_mail(body, name,html, subject, email)


In [11]:
table

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['Interactive Longon Required? (Y/N)', 'ISEN Ref #'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [96]:
df

,Type,Domain,SamAccountName,DisplayName,Description,Previous Owner employee ID,Previous Owner Name,Email,Service Account Still Required? (Y/N),Updated Owner Name,Remarks (if any),Interactive Longon Required? (Y/N),ISEN Ref #
0,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
1,description-app,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
2,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
3,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
4,interactive-logon,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
5,description-app,2458,Benjamin Grimm (Earth-616),\/Benjamin_Grimm_(Earth-616),Public Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
6,description-loc,2166,Reed Richards (Earth-616),\/Reed_Richards_(Earth-616),Public Identity,Good Characters,Brown Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
7,description-app,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
8,description-loc,2458,Benjamin Grimm (Earth-616),\/Benjamin_Grimm_(Earth-616),Public Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN
9,description-loc,2166,Reed Richards (Earth-616),\/Reed_Richards_(Earth-616),Public Identity,Good Characters,Brown Eyes,janelle.tang@au.ey.com,NaN,NaN,NaN,NaN,NaN


In [77]:
df_styled  = df.style.set_properties(**{'background-color': '#ffffbf','border':' 1pt solid #808080'}, subset=['Service Account Still Required? (Y/N)',
                                             'Updated Owner Name',
                                             'Remarks (if any)'])
df_styled

,Type,Domain,SamAccountName,DisplayName,Description,Previous Owner employee ID,Previous Owner Name,Email,Service Account Still Required? (Y/N),Updated Owner Name,Remarks (if any)
0,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,nan,nan,nan
1,description-app,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,nan,nan,nan
2,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,nan,nan,nan
3,interactive-logon,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,janelle.tang@au.ey.com,nan,nan,nan
4,interactive-logon,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,nan,nan,nan
5,description-app,2458,Benjamin Grimm (Earth-616),\/Benjamin_Grimm_(Earth-616),Public Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,nan,nan,nan
6,description-loc,2166,Reed Richards (Earth-616),\/Reed_Richards_(Earth-616),Public Identity,Good Characters,Brown Eyes,janelle.tang@au.ey.com,nan,nan,nan
7,description-app,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,nan,nan,nan
8,description-loc,2458,Benjamin Grimm (Earth-616),\/Benjamin_Grimm_(Earth-616),Public Identity,Good Characters,Blue Eyes,janelle.tang@au.ey.com,nan,nan,nan
9,description-loc,2166,Reed Richards (Earth-616),\/Reed_Richards_(Earth-616),Public Identity,Good Characters,Brown Eyes,janelle.tang@au.ey.com,nan,nan,nan
